In [53]:
import pandas as pd
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Model
from tensorflow.keras.layers import Input,Activation,Add, MaxPooling2D, Flatten, Dense
import gc

In [81]:
#Simplifies adding a layer
def Convolution(input_tensor,filters):

    x = layers.Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same', activation = 'relu')(input_tensor)

    return x

#Simple CNN model
#Conv stands for neural network layers of (x,y). x standing for the input layers and y standing for how many filter layers there are
#maxp is a pooling layer that reduces the number of nodes of the next layer
#flatten changes the previous layer to a 1 dimensional layer
#dense layers have each node connected to every node in the previous layer. In this case, we have 3 dense layers of differing weights representing
#age, gender, and race. While all previous layers have collected features, dense layers calculate based on these collected features.
#They then send their calculations to the final dense layer to determine classification
#Binary Crossentropy = Classifies based on a binary value
#Sparse Categorical Crossentropy = Classifies based on an integer value with more than 2 possible values
def Simplemodel(input_shape):

  inputs = Input((input_shape))

  conv_1= Convolution(inputs,32)

  maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)

  conv_2 = Convolution(maxp_1,64)

  maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)

  conv_3 = Convolution(maxp_2,64)

  maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)

  conv_4 = Convolution(maxp_3,64)

  maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)

  conv_5 = Convolution(maxp_4,64)

  flatten= Flatten() (conv_5)

  dense_1= Dense(64,activation='relu')(flatten)

  output_1= Dense(9,activation="relu",name='age_out')(dense_1)

  model = Model(inputs=[inputs], outputs=[output_1])

  model.compile(loss=["sparse_categorical_crossentropy"], optimizer="Adam",

  metrics=["accuracy"])

  return model

In [59]:
data = np.load('../data/raw/UnbalancedRaw/npimagearrays.npy',allow_pickle=True)
filelabels = pd.read_csv('../data/raw/UnbalancedRaw/npimagefilelabels.csv').loc[:,['file','age']]

In [60]:
filelabels.head()

,file,age
0,./Data/CroppedImages/100_0_0_20170112213500903...,9
1,./Data/CroppedImages/100_0_0_20170112215240346...,9
2,./Data/CroppedImages/100_1_0_20170110183726390...,9
3,./Data/CroppedImages/100_1_0_20170112213001988...,9
4,./Data/CroppedImages/100_1_0_20170112213303693...,9


In [61]:
X_train, X_test, y_train, y_test = train_test_split(data, filelabels, test_size=0.33, random_state=42)

In [72]:
y_testnp = np.asarray(y_test.loc[:,'age'])

In [73]:
y_trainnp = np.asarray(y_train.loc[:,'age'])

array([2, 2, 9, ..., 5, 1, 3], dtype=int64)

In [82]:
#Creating the model
Modelsimple=Simplemodel((200,200,3))
Modelsimple.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 conv2d_20 (Conv2D)          (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 100, 100, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 50, 50, 64)        3692

In [83]:
#Training the model
SimpleHistory=Modelsimple.fit(X_train,y_trainnp - 1,validation_data=(X_test,y_testnp - 1),epochs=10)

Epoch 1/10
259/497 [==============>...............] - ETA: 2:22 - loss: 7.4000 - accuracy: 0.4137

KeyboardInterrupt: 